# Лабораторная работа 5. Разработка единого шаблона предварительной обработки данных

## Подключение библиотек

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

## Загрузка данных и разделение на матрицу признаков и зависимую переменную

In [5]:
dataset = pd.read_csv('TEST_FINAL.csv')
dataset.head()

,ID,Store_id,Store_Type,Location_Type,Region_Code,Date,Holiday,Discount
0,T1188341,171,S4,L2,R3,2019-06-01,0,No
1,T1188342,172,S1,L1,R1,2019-06-01,0,No
2,T1188343,173,S4,L2,R1,2019-06-01,0,No
3,T1188344,174,S1,L1,R4,2019-06-01,0,No
4,T1188345,170,S1,L1,R2,2019-06-01,0,No


In [38]:
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, 6].values
print ("Матрица признаков"); print(X)
print ("Зависимая переменная"); print(y)

Матрица признаков
[['T1188341' 171 'S4' ... 'R3' '2019-06-01' 0]
 ['T1188342' 172 'S1' ... 'R1' '2019-06-01' 0]
 ['T1188343' 173 'S4' ... 'R1' '2019-06-01' 0]
 ...
 ['T1210603' 185 'S1' ... 'R3' '2019-07-31' 0]
 ['T1210604' 69 'S1' ... 'R4' '2019-07-31' 0]
 ['T1210605' 365 'S2' ... 'R2' '2019-07-31' 0]]
Зависимая переменная
[0 0 0 ... 0 0 0]


In [39]:
from sklearn.impute import SimpleImputer

imputer_numeric = SimpleImputer(missing_values=np.nan, strategy='mean')
X[:, [1, 6]] = imputer_numeric.fit_transform(X[:, [1, 6]])
print(X)
imputer_non_numeric = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
X[:, [2, 3, 4, 5]] = imputer_non_numeric.fit_transform(X[:, [2, 3, 4, 5]])
print(X)

[['T1188341' 171.0 'S4' ... 'R3' '2019-06-01' 0.0]
 ['T1188342' 172.0 'S1' ... 'R1' '2019-06-01' 0.0]
 ['T1188343' 173.0 'S4' ... 'R1' '2019-06-01' 0.0]
 ...
 ['T1210603' 185.0 'S1' ... 'R3' '2019-07-31' 0.0]
 ['T1210604' 69.0 'S1' ... 'R4' '2019-07-31' 0.0]
 ['T1210605' 365.0 'S2' ... 'R2' '2019-07-31' 0.0]]
[['T1188341' 171.0 'S4' ... 'R3' '2019-06-01' 0.0]
 ['T1188342' 172.0 'S1' ... 'R1' '2019-06-01' 0.0]
 ['T1188343' 173.0 'S4' ... 'R1' '2019-06-01' 0.0]
 ...
 ['T1210603' 185.0 'S1' ... 'R3' '2019-07-31' 0.0]
 ['T1210604' 69.0 'S1' ... 'R4' '2019-07-31' 0.0]
 ['T1210605' 365.0 'S2' ... 'R2' '2019-07-31' 0.0]]


### 3. Обработка пропущенных значений

### 4. Обработка категориальных данных

#### 4.1 Замена категории кодом (LabelEncoder)

In [40]:
from sklearn.preprocessing import LabelEncoder
labelencoder_y = LabelEncoder()
print("Зависимая переменная до обработки")
print(y)
y = labelencoder_y.fit_transform(y)
print("Зависимая переменная после обработки")
print(y)

Зависимая переменная до обработки
[0 0 0 ... 0 0 0]
Зависимая переменная после обработки
[0 0 0 ... 0 0 0]


#### 4.2 Применение OneHotEncoder

In [41]:
# создаем копию "грязного" объекта: спропусками и некодированными категориями
X_dirty = X.copy()
X_dirty

array([['T1188341', 171.0, 'S4', ..., 'R3', '2019-06-01', 0.0],
       ['T1188342', 172.0, 'S1', ..., 'R1', '2019-06-01', 0.0],
       ['T1188343', 173.0, 'S4', ..., 'R1', '2019-06-01', 0.0],
       ...,
       ['T1210603', 185.0, 'S1', ..., 'R3', '2019-07-31', 0.0],
       ['T1210604', 69.0, 'S1', ..., 'R4', '2019-07-31', 0.0],
       ['T1210605', 365.0, 'S2', ..., 'R2', '2019-07-31', 0.0]],
      dtype=object)

In [42]:
# Современный метод трансформации признаков
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer

# создаем список трансформеров
transformers = [
    ('onehot', OneHotEncoder(), [0]),
    ('imp', SimpleImputer(), [1,6])
]

# Создаем объект ColumnTransformer и передаем ему список трансформеров
ct = ColumnTransformer(transformers)

# Выполняем трансформацию признаков
X_transformed = ct.fit_transform(X_dirty)
print(X_transformed.shape)
X_transformed

(22265, 22267)


<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 45260 stored elements and shape (22265, 22267)>

In [44]:
# Можно преобразовать полученный многомерный массив обратно в Dataframe
X_data = pd.DataFrame(
    X_transformed,
    columns=['ID','Store_id','Store_Type','Location_Type','Region_Code','Date','Holiday','Discount'])
X_data

ValueError: Shape of passed values is (22265, 1), indices imply (22265, 8)